In [1]:
# 1 Implement a basic Statistical Machine Translation (SMT) model that uses word-by-word translation with a
# dictionary lookup approachV
import json

def load_translation_dictionary():
    """Loads a simple word-to-word translation dictionary."""
    return {
        "hello": "hola",
        "world": "mundo",
        "good": "bueno",
        "morning": "mañana",
        "how": "cómo",
        "are": "estás",
        "you": "tú",
        "is": "es",
        "this": "esto",
        "a": "un",
        "test": "prueba"
    }

def translate_sentence(sentence, translation_dict):
    """Translates a sentence using word-by-word dictionary lookup."""
    words = sentence.lower().split()
    translated_words = [translation_dict.get(word, word) for word in words]  # Keep original word if not found
    return " ".join(translated_words)

# Load dictionary
translation_dict = load_translation_dictionary()

# Example sentences
sentences = [
    "hello world",
    "good morning",
    "how are you",
    "this is a test"
]

# Perform translation
for sentence in sentences:
    translated = translate_sentence(sentence, translation_dict)
    print(f"Original: {sentence} -> Translated: {translated}")


Original: hello world -> Translated: hola mundo
Original: good morning -> Translated: bueno mañana
Original: how are you -> Translated: cómo estás tú
Original: this is a test -> Translated: esto es un prueba


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.shape[1]
        hidden = hidden.permute(1, 0, 2).expand(-1, seq_len, -1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention_weights = F.softmax(self.v(energy).squeeze(2), dim=1)
        return attention_weights

class NMTModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NMTModel, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.decoder = nn.LSTM(hidden_dim * 2, hidden_dim, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, src, tgt):
        encoder_outputs, (hidden, cell) = self.encoder(src)
        hidden = hidden.view(hidden.shape[1], -1, hidden.shape[2]).sum(dim=0)  # Adjusting hidden shape
        attn_weights = self.attention(hidden, encoder_outputs)
        context_vector = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        decoder_output, _ = self.decoder(context_vector.unsqueeze(1))
        output = self.fc(decoder_output.squeeze(1))
        return output

# Example usage
input_dim = 10
hidden_dim = 128
output_dim = 10
model = NMTModel(input_dim, hidden_dim, output_dim)

src = torch.rand(5, 10, input_dim)  # Batch of 5 sentences, 10 words each
output = model(src, src)  # Placeholder tgt input
print(output.shape)


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 2 is not equal to len(dims) = 3

In [7]:
# 4. Use a pre-trained GPT model to perform machine translation from English to French
import torch
from transformers import MarianMTModel, MarianTokenizer

def translate(text, model_name="Helsinki-NLP/opus-mt-en-fr"):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_text

english_text = ["Hello, how are you?", "This is a machine translation example."]
french_translation = translate(english_text)
print(french_translation)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

c:\Users\vinay\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinay\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-fr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

c:\Users\vinay\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

['Bonjour, comment allez-vous?', "C'est un exemple de traduction automatique."]


In [6]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\vinay\appdata\local\programs\python\python311\lib\site-packages\diamondpriceprediction-0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# 6. Generate a short poem using GPT-2 for a specific theme (e.g., "Nature")
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def generate_poem(theme, model_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    input_text = f"Write a short poem about {theme}:"
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    
    outputs = model.generate(inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
    poem = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return poem

theme = "Nature"
poem = generate_poem(theme)
print(poem)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\vinay\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinay\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

c:\Users\vinay\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write a short poem about Nature:

The poem is a poem of the nature of Nature.
...
 (The poet is the poet, and the poem the writer.)
,. (the poet and writer. The poem and


In [ ]:
# 7. Create a simple multimodal generative model that generates an image caption given an image.
import torch

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class TextGenerator(nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super(TextGenerator, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

def generate_text(model, start_token, max_length, vocab_size):
    model.eval()
    input = torch.tensor([[start_token]], dtype=torch.long)
    hidden = torch.zeros(1, 1, model.hidden_dim)
    generated_text = [start_token]
    
    for _ in range(max_length):
        output, hidden = model(input, hidden)
        probs = F.softmax(output[0, -1], dim=0)
        next_token = torch.multinomial(probs, 1).item()
        generated_text.append(next_token)
        input = torch.tensor([[next_token]], dtype=torch.long)
    
    return generated_text

def compute_reward(generated_text, target_text):
    # Simple reward function: +1 for each correct token, -1 for each incorrect token
    reward = 0
    for gen_token, tgt_token in zip(generated_text, target_text):
        if gen_token == tgt_token:
            reward += 1
        else:
            reward -= 1
    return reward

# Hyperparameters
vocab_size = 100  # Example vocab size
hidden_dim = 128
learning_rate = 0.01
max_length = 10

# Initialize model, optimizer
model = TextGenerator(vocab_size, hidden_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Example training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Generate text
    start_token = 0  # Example start token
    generated_text = generate_text(model, start_token, max_length, vocab_size)
    
    # Compute reward
    target_text = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # Example target text
    reward = compute_reward(generated_text, target_text)
    
    # Compute loss (negative reward)
    loss = -torch.tensor(reward, dtype=torch.float, requires_grad=True)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}, Reward: {reward}")


Epoch 0, Loss: 8.0, Reward: -8
Epoch 10, Loss: 10.0, Reward: -10
Epoch 20, Loss: 10.0, Reward: -10
Epoch 30, Loss: 10.0, Reward: -10
Epoch 40, Loss: 10.0, Reward: -10
Epoch 50, Loss: 10.0, Reward: -10
Epoch 60, Loss: 10.0, Reward: -10
Epoch 70, Loss: 10.0, Reward: -10
Epoch 80, Loss: 10.0, Reward: -10
Epoch 90, Loss: 10.0, Reward: -10


In [ ]:
# 7. Create a simple multimodal generative model that generates an image caption given an image.
from transformers import ViTFeatureExtractor, GPT2Tokenizer, GPT2LMHeadModel, VisionEncoderDecoderModel
import torch
from PIL import Image
import requests

# Load pre-trained models
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

def generate_caption(image_url):
    # Load and preprocess the image
    image = Image.open(requests.get(image_url, stream=True).raw)
    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values

    # Generate caption
    output_ids = model.generate(pixel_values, max_length=16, num_beams=4, early_stopping=True)
    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return caption

# Example usage
image_url = "https://images.pexels.com/photos/674010/pexels-photo-674010.jpeg?auto=compress&cs=tinysrgb&w=600"
caption = generate_caption(image_url)
print(caption)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


a close up picture of a zebra's head 


In [16]:
# 8. Demonstrate how to evaluate bias in generated content by analyzing GPT responses to prompts with
# potentially sensitive terms
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def analyze_bias(prompts, model_name="gpt2"):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    results = {}
    for prompt in prompts:
        inputs = tokenizer.encode(prompt, return_tensors="pt")
        outputs = model.generate(inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        results[prompt] = generated_text
    
    return results

# Example prompts with potentially sensitive terms
sensitive_prompts = [
    "The doctor said that the patient is",
    "The engineer is known for",
    "The nurse is always",
    "The programmer is usually"
]

bias_analysis_results = analyze_bias(sensitive_prompts)
for prompt, response in bias_analysis_results.items():
    print(f"Prompt: {prompt}\nResponse: {response}\n")


c:\Users\vinay\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain 

Prompt: The doctor said that the patient is
Response: The doctor said that the patient is in good health and is not suffering from any serious illness.

The patient was taken to the hospital for treatment.

Prompt: The engineer is known for
Response: The engineer is known for his work on the project, which has been funded by the National Science Foundation.

The project is being funded in part by a grant from the U.S. Department of Energy's Office of Science and Technology Policy.

Prompt: The nurse is always
Response: The nurse is always on the phone with the patient, and she's always there to help.

"She's a very good person," said Dr. David L. Lippman, a pediatrician at the University of California, San Francisco

Prompt: The programmer is usually
Response: The programmer is usually a programmer, but he or she is often a software engineer.

The software developer is a person who is responsible for the development of the software. The software developers are usually responsible to th

In [22]:
# 9. Create a simple Neural Machine Translation model with PyTorch for translating English phrases to German.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class SimpleNMT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNMT, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, src, tgt):
        encoder_outputs, (hidden, cell) = self.encoder(src)
        output, _ = self.decoder(tgt, (hidden, cell))
        output = self.fc(output)
        return output

# Hyperparameters
input_dim = 10 
hidden_dim = 128
output_dim = 10  
learning_rate = 0.01
num_epochs = 100

# Initialize model, loss function, optimizer
model = SimpleNMT(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Example training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Example input and target data
    src = torch.rand(5, 10, input_dim)  # Batch of 5 sentences, 10 words each
    tgt = torch.rand(5, 10, hidden_dim)  # Adjusted target shape to match decoder input
    tgt_labels = torch.randint(0, output_dim, (5, 10))  # Random target labels
    
    # Forward pass
    output = model(src, tgt)
    
    # Compute loss
    loss = criterion(output.view(-1, output_dim), tgt_labels.view(-1))
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")


Epoch 0, Loss: 2.311655044555664
Epoch 10, Loss: 2.28181529045105
Epoch 20, Loss: 2.3117854595184326
Epoch 30, Loss: 2.294337749481201
Epoch 40, Loss: 2.2987985610961914
Epoch 50, Loss: 2.296395778656006
Epoch 60, Loss: 2.300971746444702
Epoch 70, Loss: 2.2998533248901367
Epoch 80, Loss: 2.301016092300415
Epoch 90, Loss: 2.3049814701080322
